# Functional MRI pial flattening in 20 minutes

This notebook demonstrates how to use PyCortex to:
1. Cut and flatten brain surfaces in under 20 minutes;
2. Preview flattened surface in under 10 seconds;
3. Visualize BOLD fMRI data on the flattened surfaces

## Prerequisites

You will need to run auto

#### 1. Reconcile subject's anatomy with Freesurfer

Make sure you have fully reconciled the subject's. Detailed guidance is out of scope for this notebook, but roughly you should've run these lines:

```
recon-all -autorecon1 -s sub-01
recon-all -autorecon2 -s sub-01
recon-all -autorecon3 -s sub-01
```

#### 2. Install Python dependencies

In [13]:
import cortex
import nibabel as nib
import numpy as np
from matplotlib import pyplot as plt

## Step 1: Align Function MRI to Anatomical

`cortex.align.automatic` perform automatic alignment using Freesurfer's boundary-based registration. It is fully automated. This should take ~1 minute.

In [ ]:
bold_nii_gz_path = "path to bold.nii.gz"
cortex.align.automatic(
    subject="sub-01",
    xfmname="full",
    reference=bold_nii_gz_path,
)

## Step 2: Surface Cutting and Flattening

`cortex.segment.cut_surface` performs surface cutting. It is semi-automatic, meaning it requires a bit of manual work. It will start Blender for us to manually specify the cuts. Make sure Blender is installed on your system.

#### A. Left hemisphere

Let's start with the left hemisphere.

Please follow [this guide](https://www.youtube.com/watch?v=D4tylQ_mMuM) on cuts. This will take ~15 minutes.

In [ ]:
cortex.segment.cut_surface(
    "sub-01",
    "lh",
    name="exp",
    flatten_with="blender",
    method="CONFORMAL",
    recache=True,
    do_import_subject=False,
)

#### B. Right hemisphere

Then repeat the same for the right hemisphere.

This will take another ~5 minutes.


In [ ]:
# Right hemisphere
cortex.segment.cut_surface(
    "sub-01",
    "rh",
    name="exp",
    flatten_with="blender",
    method="CONFORMAL",
    recache=True,
    auto_overwrite=True,
    do_import_subject=True,
)

## Step 3: Visualize fMRI

Now we'll load the BOLD fMRI data and prepare it for visualization. We'll compute the temporal mean of the BOLD signal to create a static visualization.

Then we can use `cortex.quickshow` to visualize the BOLD data on the flattened surfaces using pycortex.

This will take 30 seconds.

In [ ]:
# Load functional data
bold_path = "path to bold.nii.gz"
img = nib.load(bold_path)
data = img.get_fdata()

# Compute temporal mean
mean_vol = np.mean(data, axis=3)

# Transpose to match expected orientation (96,96,76) → (76,96,96)
mean_vol = np.transpose(mean_vol, (2, 1, 0))

# Select cuboid
cuboid = np.array([[0, 20, 0], [50, 70, 70]])  # (2, 3) as a pair of (min, max) voxels
print(mean_vol.shape)
cuboid_slice = tuple(slice(*span) for span in cuboid.transpose())
mean_vol_filtered = np.zeros_like(mean_vol)
mean_vol_filtered[cuboid_slice] = mean_vol[cuboid_slice]
mean_vol = mean_vol_filtered


# Create Volume object for visualization
vol = cortex.Volume(
    mean_vol,
    subject="sub-01",
    depth=1,
    height=512,
    xfmname="full",
    vmin=np.min(mean_vol),
    vmax=np.max(mean_vol),
)

# Display the visualization
cortex.quickshow(vol, with_colorbar=True, recache=True)
plt.show()